# building deep neural network(step by step)

this is image classification task  
- in this part will use relu to improve our model.
- build a deeper neural network (more than 1 hidden layer)
- implement an easy-to-use neural network class


In [28]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams["figure.figsize"]=(5.0,4.0)
plt.rcParams["image.interpolation"]='nearest'
plt.rcParams['image.cmap']='gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 3-initialization

- first function to initialization two layer model
- generalize this initialization process to L layers.
  
  model's structure is LINEAR->RELU->LINEAR->SIGMOID

In [29]:
def initialize_parameters(n_x,n_h,n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    returns:
    parameters: -- python dictionary :
       W1 -- weight matrix of shape(n_h,n_x)
       b1 -- bias vector of shape(n_h,1)
       W2 -- weight matrix of shape (n_y,n_h)
       b2 -- bias vector of shape(n_y,1)
    """
    #.....e ...
    np.random.seed(1)
    W1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    
    #W1=np.array([n_h,n_x])
    #b1=np.array([n_h,1])
    #W2=np.array([n_y,n_h])
    #b2=np.array([n_y,1])
    
    assert(W1.shape==(n_h,n_x))
    assert(b1.shape==(n_h,1))
    assert(W2.shape==(n_y,n_h))
    assert(b2.shape==(n_y,1))
    
    parameters={
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    return parameters

In [30]:
parameters=initialize_parameters(2,2,1)
print("W1= ",parameters["W1"])
print("b1= ",parameters["b1"])
print("W2= ",parameters["W2"])
print("b2= ",parameters["b2"])

W1=  [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1=  [[ 0.]
 [ 0.]]
W2=  [[ 0.00865408 -0.02301539]]
b2=  [[ 0.]]


## 3.2 L-layer Neural Network

In [31]:
def initialize_parameters_deep(layer_dims):
    """
    arguments:
    layer_dims -- python array(list) constaining the dimensions of 
    each layer in our network
    returns:
    W1 --
    b1 --
    """
    np.random.seed(3)
    parameters={}
    L=len(layer_dims) #number of layers in the network
    print("L ",L)
    for l in range(1,L):
        parameters['W'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)]=np.zeros((layer_dims[l],1))
        print('l',l)
        assert(parameters['W'+str(l)].shape==(layer_dims[l],layer_dims[l-1]))
        assert(parameters['b'+str(l)].shape==(layer_dims[l],1))
    return parameters
        

In [32]:
#[5,4,3]is stand for there are 3 hidden layers.
parameters=initialize_parameters_deep([5,4,3])
print('W1= '+str(parameters["W1"]))
print('b1= '+str(parameters["b1"]))
print('W2= '+str(parameters["W2"]))
print('b2= '+str(parameters["b2"]))


L  3
l 1
l 2
W1= [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1= [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
W2= [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2= [[ 0.]
 [ 0.]
 [ 0.]]


# forward propagation module
## 4.1 linear forward

In [46]:
def linear_forward(A,W,b):
    """
    linear part in forward propagation
    argument:
    A -- activations from previous layers(or inputs ):size of layers, or 
        size of inputs
    W -- weights matrix (size of current layer, the last layer size)
    b -- bias (size of current layer,1)
    returns:
    Z -- the input of the activation function,(pre-activation parameter)
    cache -- A,W,b
    """
    #fuck...
    #Z=np.random.randn(W.shape[0],A.shape[1])*0.01
    Z=np.dot(W,A)+b
    assert(Z.shape==(W.shape[0],A.shape[1]))
    cache=(A,W,b)
    return Z,cache

In [53]:
A,W,b=linear_forward_test_case()
Z,linear_cache=linear_forward(A,W,b)
print("A: ",linear_cache[0].shape)
print("W: ",linear_cache[1].shape)
print("b: ",linear_cache[2].shape)
print("Z= "+str(Z))
print(Z.shape)

A:  (3, 2)
W:  (1, 3)
b:  (1, 1)
Z= [[ 3.26295337 -1.23429987]]
(1, 2)


## 4.2 linear-activation forward

In [54]:
def linear_activation_forward(A_prew,W,b,activation):
    """
    Implement the forward propagation for linear -> activation layer
    Arguments:
    A_prew --activations from previous layer
    W -- weights matrix: 
    b -- bias vector ,
    activation -- relu or sigmoid 
    returns:
    A -- the output of the activation function,(post-activation value)
    cache -- a python dictionary  "linear_cache" and "cactivation_cache"
      stored for computing the backward pass efficiently 
    """
    if activation=="sigmoid":
        Z,linear_cache=linear_forward(A_prew,W,b)
        A,activation_cache=sigmoid(Z)
    if activation=="relu":
        Z,linear_cache=linear_forward(A_prew,W,b)
        A,activation_cache=relu(Z)
    assert(A.shape==(W.shape[0],A_prev.shape[1]))
    cache=(linear_cache,activation_cache)
    return A,cache
    
    

In [55]:
A_prew,W,b=linear_activation_forward_test_case()

NameError: name 'linear_activation_forward_test_case' is not defined